In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

In [33]:
### 함수 정의 ###

# 수치형 데이터 다운캐스팅(Downcasting)
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name=df[col].dtype.name
        if dtype_name=='object':
            pass
        elif dtype_name=='bool':
            df[col]=df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round()==df[col]).all():
            df[col]=pd.to_numeric(df[col], downcast='integer')
        else:
            df[col]=pd.to_numeric(df[col], downcast='float')
    end_mem=df.memory_usage().sum() / 1024**2
    
    if verbose:
        print('{:.1f}% 압축됨'.format(100*(start_mem-end_mem)/start_mem))

# train/test 데이터프레임과 stores 데이터프레임을 입력받아 데이터 머지 후, 반환하는 함수
def train_store_mearge(input, stores):
    return pd.merge(input, stores, how='left', on='store_nbr')

# oil 데이터의 빠진 날짜와 dcoilwtico 데이터를 interpolation으로 채운 dataframe을 반환하는 함수
def fill_oil_data(df):
    # date 열을 인덱스로 설정
    df.set_index('date', inplace=True)
    # 전체 날짜 범위를 생성하여 누락된 날짜를 추가 (min에서 max 사이의 모든 날짜)
    df = df.reindex(pd.date_range(start=df.index.min(), end=df.index.max()))
    # price 값을 선형 보간법으로 채우기
    df['dcoilwtico'] = df['dcoilwtico'].interpolate()
    # 인덱스를 다시 date 열로 복원
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'date'}, inplace=True)
    return df

# 날짜와 holiday list를 입력받아, 해당 날짜가 holiday 인지 판별하여 True/False 값 반환하는 함수
def is_holiday(dt, hdts):
    result = dt in hdts.values
    return result

# 날짜를 입력받아 해당 날짜가 workday 인지 weekend인지 판별하고 결과를 'workday' or 'weekend'로 반환하는 함수
def normal_day_type(dt):
    if dt.weekday() >= 5:
        return 'Weekend'
    else:
        return 'Work Day'
    
# 이벤트/Holiday 이름에서 Transfer를 삭제하는 함수
def replace_prefix(value):
    # 접두사가 "Traslado "로 시작하는지 확인
    if value.startswith("Traslado "):
        # 접두사를 빈 문자열로 교체
        return value.replace("Traslado ", "", 1)
    return value
    
# ['date'], ['city'], ['state'] 그리고 df_holiday_events 데이터프레임을 입력받아 date.holiday type을 반환하는 함수
def date_type(dt, city, state, dhe):
    dt_type = ""
    dt_description = ""
    # 입력받은 날짜가 df_holiday_events에 포함되어 있는 경우, lcoal->regional->national holidays 순서로 해당되는 type을 확인
    if is_holiday(dt, dhe['date']):
        holidays = dhe[dhe['date'] == dt]
        for _, holiday in holidays.iterrows():
            # Local인 경우, city 비교
            if holiday['locale'] == 'Local' and city == holiday['locale_name']:
                dt_type = holiday['type']
                dt_description = replace_prefix(holiday['description'])
                break
            elif holiday['locale'] == 'Regional' and state == holiday['locale_name']:
                dt_type = holiday['type']
                dt_description = replace_prefix(holiday['description'])
            elif holiday['locale'] == 'National' and dt_type == "":
                dt_type = holiday['type']
                dt_description = replace_prefix(holiday['description'])
    if dt_type == "":
        dt_type = normal_day_type(dt)
        if dt_type =='Weekend':
            dt_description = "Weekend"
        else:
            dt_description = "Work Day"
    return (dt_type, dt_description)

def pre_proecess(df, df_stores, df_holiday_events, df_oil, purpose='train'):
    # df_oil_filled 데이터의 빠진 날짜 및 dcoilwtico 데이터 채우기
    df_oil_filled = df_oil.copy()
    df_oil_filled = fill_oil_data(df_oil_filled)
    # 데이터의 첫 날 (2013-01-01)의 dcoilwtico 데이터는 누락되어 있으므로, 그 다음날의 dcoilwtico 값과 동일한 것으로 가정
    df_oil_filled.iloc[0, 1] = df_oil_filled.iloc[1, 1]  
    # train 데이터에 dcoilwtico 값 추가
    df_train_stores = train_store_mearge(df, df_stores)

    # 헤더만 쓰기 위해 첫 번째 빈 데이터프레임을 저장
    with open ('temp_data.csv', 'w') as f:
        pd.DataFrame(columns=df_train_stores.columns.tolist()+['date_type']+['date_description']+['year']+['month']+['day']+['weekday']).to_csv(f, index=False)

    # 데이터 프레임을 chunk_size씩 데이터를 확인하면서, holiday_events 데이터를 기준으로 date_type을 확인해서 train 데이터에 추가
    chunk_size = 1000
    end_row = len(df_train_stores)

    for i in range(0, end_row + chunk_size, chunk_size):
        if i > end_row:
            print(f"{idx} rows are porcessed.")
            break
        else:
            chunk = df_train_stores.iloc[i:i + chunk_size].copy()  # i에서 i + chunk_size까지 슬라이싱
            # date_type을 구분
            for idx, row in chunk.iterrows():
                dt_tp, dt_des = date_type(row['date'], row['city'], row['state'], df_holiday_events)
                chunk.at[idx,'date_type']=dt_tp
                chunk.at[idx,'date_description']=dt_des
        # chunk['date']를 year, month, day, weekday로 분리
        chunk['date'] = pd.to_datetime(chunk['date'])
        chunk['year'] = chunk['date'].dt.year
        chunk['month'] = chunk['date'].dt.month
        chunk['day'] = chunk['date'].dt.day
        chunk['weekday'] = chunk['date'].dt.day_name()

        print(f"{i+1} row(s) are porcessed.")
        # 결과를 파일에 추가 모드로 저장
        chunk.to_csv('temp_data.csv', mode='a', header=False, index=False)

    # train data에 stroes 및 date_type 정보가 추가된 데이터를 temp_data 데이터프레임에 임시 저장
    temp_data = pd.read_csv('temp_data.csv', parse_dates =['date'])
    # temp_data에 df_oil_filled 데이터를 기준으로 dcoilwtico 데이터를 추가해서, train_data 데이터 프레임에 저장
    train_data = pd.merge(temp_data, df_oil_filled, how='left', on='date')
    
    # 처리한 데이터를 저장할 csv 파일명을 purpose 기준으로 생성하고, train_data를 해당 csv 파일에 저장
    filename = purpose+".csv"
    train_data.to_csv(filename, index=False)

    print("Data preprocess is completed, and the result is saved to train_data.csv.")
    print(train_data.info())

    # temp_data.csv 파일 삭제
    if os.path.exists('temp_data.csv'):
        os.remove('temp_data.csv')
        print(f"temp_data.csv 파일이 삭제되었습니다.")
    else:
        print(f"temp_data.csv 파일이 존재하지 않습니다.")

    return True

# 모델에 사용할 수 있도록 데이터 변환을 위한 함수 정의

from sklearn.preprocessing import LabelEncoder

def label_encoding(df1,df2,features = ['year', 'month', 'weekday','date_description', 'store_nbr','family', 'city', 'state', 'type', 'cluster']):

    # LabelEncoder 객체 생성
    label_encoder = LabelEncoder()
    
    # 문자열 변수들을 정수로 인코딩
    for ft in features:
        label_encoder.fit(df1[ft])
        df1[ft]=label_encoder.transform(df1[ft])
        df2[ft]=label_encoder.transform(df2[ft])

    return df1, df2

from sklearn.preprocessing import StandardScaler
def scaler_function (df1, df2, features = ['onpromotion','dcoilwtico']):
    # StandardScaler 생성
    scaler = StandardScaler()
    scaler.fit(df1[features])

    # 선택한 열만 표준화
    df1[features] = scaler.transform(df1[features])
    df2[features] = scaler.transform(df2[features])

    # 변환된 데이터를 DataFrame으로 반환
    return df1, df2


In [12]:
# train, store, holiday_events, oil, test 데이터 로드 / df_test 데이터에 대한 transaction 데이터는 존재하지 않기 때문에 transcation 데이터는 모델 학습에 사용하지 않음
path = 'store-sales-time-series-forecasting/'
df_train = pd.read_csv(path + 'train.csv',parse_dates =['date'])
df_stores = pd.read_csv(path + 'stores.csv')
df_holiday_events = pd.read_csv(path + 'holidays_events.csv',parse_dates =['date'])
df_oil = pd.read_csv(path + 'oil.csv',parse_dates =['date'])
df_test = pd.read_csv(path + 'test.csv',parse_dates =['date'])

In [14]:
# 원본 데이터 파일을 merge 및 1차 전처리
pre_proecess(df = df_train, df_stores=df_stores, df_holiday_events=df_holiday_events, df_oil=df_oil, purpose='train_w_val')
pre_proecess(df = df_test, df_stores=df_stores, df_holiday_events=df_holiday_events, df_oil=df_oil, purpose='test')


1 row(s) are porcessed.
1001 row(s) are porcessed.
2001 row(s) are porcessed.
3001 row(s) are porcessed.
4001 row(s) are porcessed.
5001 row(s) are porcessed.
6001 row(s) are porcessed.
7001 row(s) are porcessed.
8001 row(s) are porcessed.
9001 row(s) are porcessed.
10001 row(s) are porcessed.
11001 row(s) are porcessed.
12001 row(s) are porcessed.
13001 row(s) are porcessed.
14001 row(s) are porcessed.
15001 row(s) are porcessed.
16001 row(s) are porcessed.
17001 row(s) are porcessed.
18001 row(s) are porcessed.
19001 row(s) are porcessed.
20001 row(s) are porcessed.
21001 row(s) are porcessed.
22001 row(s) are porcessed.
23001 row(s) are porcessed.
24001 row(s) are porcessed.
25001 row(s) are porcessed.
26001 row(s) are porcessed.
27001 row(s) are porcessed.
28001 row(s) are porcessed.
29001 row(s) are porcessed.
30001 row(s) are porcessed.
31001 row(s) are porcessed.
32001 row(s) are porcessed.
33001 row(s) are porcessed.
34001 row(s) are porcessed.
35001 row(s) are porcessed.
36001

((2945646, 16), (2945646,), (55242, 16), (55242,))

In [30]:

# train_w_val 데이터를 훈련 목적의 train_data (2017-07-16 이전)과 validation data (2017-07-16 ~ 08.15)로 분리
df_train_w_val = pd.read_csv('train_w_val.csv',parse_dates =['date'])
train_idx = pd.to_datetime(df_train_w_val['date']) < pd.to_datetime('2017-07-16')
val_idx = pd.to_datetime(df_train_w_val['date']) >= pd.to_datetime('2017-07-16')

# train, validation 및 test 데이터를 분리해서 dataframe에 저장
train_data = df_train_w_val[train_idx].drop('sales', axis=1)
train_target = df_train_w_val[train_idx]['sales']
val_data = df_train_w_val[val_idx].drop('sales', axis=1)
val_target = df_train_w_val[val_idx]['sales']
test_data = pd.read_csv('test.csv')

# 반복 모델 훈련 및 검증, 테스트를 위한 데이터를 /TestData 폴더에 csv 파일로 저장
train_data.to_csv('TestData/train_data.csv', index=False)
train_target.to_csv('TestData/train_target.csv', index=False)
val_data.to_csv('TestData/val_data.csv', index=False)
val_target.to_csv('TestData/val_target.csv', index=False)
test_data.to_csv('TestData/test_data.csv', index=False)


In [32]:
# 모델 별로 사용가능한 데이터를 One-Hot Encoding, StandardScale 및 Label Encoding을 적용하여, 별도 csv 파일로 저장장

# class type data
features = ['year', 'month', 'weekday','date_description', 'store_nbr','family', 'city', 'state', 'type', 'cluster']

# One-hot encoding 수행
train_data_oh = pd.get_dummies(train_data, columns=features)
val_data_oh = pd.get_dummies(val_data, columns=features)
test_data_oh = pd.get_dummies(test_data, columns=features)

# Numeric Data 표준화 (train data로 학습한 모델을 val & test 데이터에 적용)
train_data_oh_scaled, val_data_oh_scaled = scaler_function(train_data_oh, val_data_oh)
train_data_oh_scaled, test_data_oh_scaled = scaler_function(train_data_oh, test_data_oh)

train_data_oh_scaled.to_csv('TestData/train_data_oh_scaled.csv', index=False)
val_data_oh_scaled.to_csv('TestData/val_data_oh_scaled.csv', index=False)
test_data_oh_scaled.to_csv('TestData/test_data_oh_scaled.csv', index=False)

# label Encoding 수행 (Train data로 학습한 encoding을 val과 test에 적용)
train_data_label_encoded, val_data_label_encoded = label_encoding(train_data, val_data)
train_data_label_encoded, test_data_label_encoded = label_encoding(train_data, test_data)
train_data_label_encoded.to_csv('TestData/train_data_label_encoded.csv', index=False)
val_data_label_encoded.to_csv('TestData/val_data_label_encoded.csv', index=False)
test_data_label_encoded.to_csv('TestData/test_data_label_encoded.csv', index=False)


